### ライブラリインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
import os
import sys
import cv2
import glob
os.makedirs('output/summary', exist_ok=True)

### 関数

In [2]:
def aliasing(targ,K):
    filter_aliasing=K<np.max(K)/8
    return targ*filter_aliasing

def vortex_detect(Psi,H):
    a=int(len(Psi)/2)
    filter_den=(((np.abs(psi)**2*(1-H))/np.max((np.abs(psi)**2*(1-H))))[a:-1,a:-1])<0.1
    phase=(np.log(Psi).imag*(1-H))[a:-1,a:-1]
    fil_1=np.roll(phase,1,axis=0)
    fil_2=np.roll(fil_1,1,axis=1)
    fil_3=np.roll(fil_2,-1,axis=0)
    fil_4=np.roll(fil_3,-1,axis=0)
    fil_5=np.roll(fil_4,-1,axis=1)
    fil_6=np.roll(fil_5,-1,axis=1)
    fil_7=np.roll(fil_6,1,axis=0)
    fil_8=np.roll(fil_7,1,axis=0)
    filter_po_1=(fil_1>fil_2)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_2=(fil_8>fil_1)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_3=(fil_8>fil_1)*(fil_1>fil_2)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_4=(fil_8>fil_1)*(fil_1>fil_2)*(fil_2>fil_3)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_5=(fil_8>fil_1)*(fil_1>fil_2)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_5>fil_6)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_6=(fil_8>fil_1)*(fil_1>fil_2)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_6>fil_7)*(fil_7>fil_8)
    filter_po_7=(fil_8>fil_1)*(fil_1>fil_2)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_7>fil_8)
    filter_po_8=(fil_8>fil_1)*(fil_1>fil_2)*(fil_2>fil_3)*(fil_3>fil_4)*(fil_4>fil_5)*(fil_5>fil_6)*(fil_6>fil_7)
    detected_vortex_po=(-1)*-2.0*(filter_po_1+filter_po_2+filter_po_3+filter_po_4+filter_po_5+filter_po_6+filter_po_7+filter_po_8)*filter_den
    filter_ne_1=(fil_1<fil_2)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_2=(fil_8<fil_1)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_3=(fil_8<fil_1)*(fil_1<fil_2)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_4=(fil_8<fil_1)*(fil_1<fil_2)*(fil_2<fil_3)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_5=(fil_8<fil_1)*(fil_1<fil_2)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_5<fil_6)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_6=(fil_8<fil_1)*(fil_1<fil_2)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_6<fil_7)*(fil_7<fil_8)
    filter_ne_7=(fil_8<fil_1)*(fil_1<fil_2)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_7<fil_8)
    filter_ne_8=(fil_8<fil_1)*(fil_1<fil_2)*(fil_2<fil_3)*(fil_3<fil_4)*(fil_4<fil_5)*(fil_5<fil_6)*(fil_6<fil_7)
    detected_vortex_ne=-2.0*(filter_ne_1+filter_ne_2+filter_ne_3+filter_ne_4+filter_ne_5+filter_ne_6+filter_ne_7+filter_ne_8)*filter_den
    detected_vortex=detected_vortex_ne+detected_vortex_po
    pos_po=[]
    pos_ne=[]
    for i in range(len(detected_vortex_po)):
        for j in range(len(detected_vortex_po)):
            if detected_vortex_po[i,j]!=0:
                pos_po.append([i,j])
    pos_po=np.array(pos_po)
    delete_list=[]
    for i in range(len(pos_po)-1):
        for j in range(i+1,len(pos_po)):
            if (pos_po[i][0]-pos_po[j][0])**2+(pos_po[i][1]-pos_po[j][1])**2<8**2:
                if j not in delete_list:
                    delete_list.append(j)
    pos_po=np.delete(pos_po,delete_list,0)
    for i in range(len(detected_vortex_ne)):
        for j in range(len(detected_vortex_ne)):
            if detected_vortex_ne[i,j]!=0:
                pos_ne.append([i,j])
    pos_ne=np.array(pos_ne)
    delete_list=[]
    for i in range(len(pos_ne)-1):
        for j in range(i+1,len(pos_ne)):
            if (pos_ne[i][0]-pos_ne[j][0])**2+(pos_ne[i][1]-pos_ne[j][1])**2<8**2:
                if j not in delete_list:
                    delete_list.append(j)
    pos_ne=np.delete(pos_ne,delete_list,0)
    del delete_list
    delete_list_po=[]
    delete_list_ne=[]
    for i in range(len(pos_po)):
        for j in range(len(pos_ne)):
            if (pos_po[i][0]-pos_ne[j][0])**2+(pos_po[i][1]-pos_ne[j][1])**2<8**2:
                delete_list_po.append(i)
                delete_list_ne.append(j)
                break
    pos_po=np.delete(pos_po,delete_list_po,0)
    pos_ne=np.delete(pos_ne,delete_list_ne,0)
    
    moment_x=0
    moment_y=0
    for i in range(len(pos_po)):
        moment_x=moment_x+(pos_po[i][0]-len(phase)/2)
        moment_y=moment_y+(pos_po[i][1]-len(phase)/2)
    for i in range(len(pos_ne)):
        moment_x=moment_x-(pos_ne[i][0]-len(phase)/2)
        moment_y=moment_y-(pos_ne[i][1]-len(phase)/2)
    moment=(moment_x**2+moment_y**2)**0.5*0.077
    
    return len(pos_po)+len(pos_ne),moment,pos_po,pos_ne

def spectrum(target,filter_list):
    theta=np.log(target).imag
    v_x=np.fft.ifft2(-1j*k_x*np.fft.fft2(theta))
    v_y=np.fft.ifft2(-1j*k_y*np.fft.fft2(theta))
    density_sqrt=np.abs(target)
    
    Energy_kin_density=(np.abs(np.fft.fft2(density_sqrt*v_x))**2+np.abs(np.fft.fft2(density_sqrt*v_y))**2)
    Energy_kin_spe=np.zeros(len(filter_list))
    for j in range(len(filter_list)):
        Energy_kin_spe[j]=(np.sum(Energy_kin_density*filter_list[j])/np.sum(filter_list[j]))
    
    k_sq_tmp=k_sq.copy()
    k_sq_tmp[0,0]=10**(-10)
    scalar_potential_k=-(-1j*k_x*np.fft.fft2(density_sqrt*v_x)-1j*k_y*np.fft.fft2(density_sqrt*v_y))/(k_sq_tmp)
    #密度含めた定義にしてある
    u_compressible_x_k=-1j*k_x*scalar_potential_k
    u_compressible_y_k=-1j*k_y*scalar_potential_k
    Energy_kin_compressible_density=(np.abs(u_compressible_x_k)**2+np.abs(u_compressible_y_k)**2)
    Energy_kin_compressible_spe=np.zeros(len(filter_list))
    for j in range(len(filter_list)):
        Energy_kin_compressible_spe[j]=(np.sum(Energy_kin_compressible_density*filter_list[j])/np.sum(filter_list[j]))
    
    u_incompressible_x_k=np.fft.fft2(density_sqrt*v_x)-u_compressible_x_k
    u_incompressible_y_k=np.fft.fft2(density_sqrt*v_y)-u_compressible_y_k
    Energy_kin_incompressible_density=(np.abs(u_incompressible_x_k)**2+np.abs(u_incompressible_y_k)**2)
    Energy_kin_incompressible_spe=np.zeros(len(filter_list))
    for j in range(len(filter_list)):
        Energy_kin_incompressible_spe[j]=(np.sum(Energy_kin_incompressible_density*filter_list[j])/np.sum(filter_list[j]))
    
    return Energy_kin_spe,Energy_kin_compressible_spe,Energy_kin_incompressible_spe

### 空間生成

In [3]:
with open('config.yaml') as file:
    object_yaml = yaml.safe_load(file)
    space_div_num=object_yaml['space_div_num']
    a_osc_num=object_yaml['a_osc_num']
    delta_t=object_yaml['delta_t']
    t_N=object_yaml['t_N']
    vortex_num=object_yaml['vortex_num']
    R_0=object_yaml['R_0']
    V_0=object_yaml['V_0']
    output_wavefunction=object_yaml['output_wavefunction']
    filter_bin=object_yaml['filter_bin']
delta_x=1/a_osc_num
Nonlinearity=125**(1/2)*10**4

space=np.linspace(-delta_x*space_div_num/2,delta_x*(space_div_num-2)/2,space_div_num)
x,y=np.meshgrid(space,space)
space_sq=x**2+y**2
k=2*np.pi*np.fft.fftfreq(space_div_num,d=delta_x)
k_x,k_y=np.meshgrid(k,k)
k_sq=k_x**2+k_y**2
print("Spatial resolution : "+str(delta_x))
print("Wavenumber resolution : "+str(k[1]))
print("Time resolution : "+str(delta_t))

H=(x-np.min(x)/2)**2+(y-np.min(y)/2)**2>(R_0*0.6*0.9)**2
H=H*((x+np.min(x)/2)**2+(y-np.min(y)/2)**2>(R_0*0.6*0.9)**2)
H=H*((x-np.min(x)/2)**2+(y+np.min(y)/2)**2>(R_0*0.6*0.9)**2)
H=H*((x+np.min(x)/2)**2+(y+np.min(y)/2)**2>(R_0*0.6*0.9)**2)
V=(np.tanh(0.6*(((x-np.min(x)/2)**2+(y-np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x+np.min(x)/2)**2+(y-np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x-np.min(x)/2)**2+(y+np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x+np.min(x)/2)**2+(y+np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*V_0

Spatial resolution : 0.14285714285714285
Wavenumber resolution : 0.04295146206079795
Time resolution : 0.01


### 解析

In [4]:
path_list=glob.glob("output/wavefunction/*")
#path_list=glob.glob("2020_6_30/virgo/test1/wavefunction/*")
dipole_moment_list=np.zeros(len(path_list))
vortex_number_list=np.zeros(len(path_list))
all_kinetic_energy_list=np.zeros(len(path_list))
compressible_kinetic_energy_list=np.zeros(len(path_list))
incompressible_kinetic_energy_list=np.zeros(len(path_list))
Z=np.linspace(0,int(np.sqrt(np.max(k_sq))/2),int(np.sqrt(np.max(k_sq))/(2*filter_bin)+1))
filter_list=[]
for j in range(int(np.sqrt(np.max(k_sq))/(2*filter_bin))+1):
    if j==0:
        filter_tmp=np.sqrt(k_sq)<=filter_bin/2
        filter_tmp*=np.sqrt(k_sq)>0
        filter_list.append(1*filter_tmp)
    else:
        filter_tmp=np.sqrt(k_sq)<=j/int(1/filter_bin)+filter_bin/2
        filter_tmp*=np.sqrt(k_sq)>j/int(1/filter_bin)-filter_bin/2
        filter_list.append(1*filter_tmp)
        
for i in tqdm(range(len(path_list))):
#for i in tqdm(range(10)):
    if i==0:
        psi=np.load("output/wavefunction/initial.npy")
    else:
        psi=np.load("output/wavefunction/"+str(output_wavefunction*i)+".npy")
    
    E_kin_spe,E_kin_compressible_spe,E_kin_incompressible_spe=spectrum(psi,filter_list)
    all_kinetic_energy_list[i]=np.sum(E_kin_spe)
    compressible_kinetic_energy_list[i]=np.sum(E_kin_compressible_spe)
    incompressible_kinetic_energy_list[i]=np.sum(E_kin_incompressible_spe)
    
    all_vortex_number,d,vortex_position_positive,vortex_position_negative=vortex_detect(psi,H)
    if all_vortex_number==0:
        dipole_moment_list[i]=0
    else:
        dipole_moment_list[i]=d/all_vortex_number
    vortex_number_list[i]=all_vortex_number
    time=np.linspace(0,i,i+1)
    
##############################
#####     表示図設定     #####
##############################

    fig = plt.figure(figsize=(30,20))
    fig.suptitle("t="+str(i))
    ax1 = fig.add_subplot(3, 4, 1)
    ax2 = fig.add_subplot(3, 4, 2)
    ax3 = fig.add_subplot(3, 4, 3)
    ax4 = fig.add_subplot(3, 4, 4)
    ax5 = fig.add_subplot(3, 3, 4)
    ax6 = fig.add_subplot(3, 3, 5)
    ax7 = fig.add_subplot(3, 3, 6)
    ax8 = fig.add_subplot(3, 3, 7)
    ax9 = fig.add_subplot(3, 3, 8)
    ax10 = fig.add_subplot(3, 3, 9)
#密度表示
    ax1.set_title("density")
    ax1.imshow((np.abs(psi)**2)[int(np.shape(psi)[0]/2):,int(np.shape(psi)[0]/2):],origin='lower')
#検知した渦表示
    ax2.set_title("detected vortices")
    ax2.imshow(H[int(np.shape(psi)[0]/2):,int(np.shape(psi)[0]/2):],cmap="binary",origin='lower')
    for j in range(len(vortex_position_positive)):
        ax2.plot(vortex_position_positive[j][1],vortex_position_positive[j][0],marker='o',ms=2,color="blue")
    for j in range(len(vortex_position_negative)):
        ax2.plot(vortex_position_negative[j][1],vortex_position_negative[j][0],marker='o',ms=2,color="red")
#渦の数表示
    ax3.set_title("vortex_number")
    ax3.set_xlim(0,len(path_list)-1)
    ax3.set_xlabel('t (ħ/(gn))')
    ax3.set_ylim(0,vortex_num)
    ax3.set_ylabel('N_vortex')
    ax3.plot(time,vortex_number_list[0:i+1])
#双極子モーメント表示
    ax4.set_title("dipole_moment")
    ax4.set_xlim(0,len(path_list)-1)
    ax4.set_xlabel('t (ħ/(gn))')
    ax4.set_ylim(-1,5)
    ax4.set_ylabel('d')
    ax4.plot(time,dipole_moment_list[0:i+1])
#非圧縮部分運動エネルギースペクトル表示
    ax5.set_title("incompressible kinetic energy spectrum")
    ax5.set_xscale('log')
    ax5.set_xlabel('k')
    ax5.set_yscale('log')
    ax5.set_ylabel('E_ic(k)')
    #ax5.set_ylim(10**2,10**5)
    ax5.plot(Z,E_kin_incompressible_spe)
    #ax5.plot(Z,Z**(-5/3))
    #ax5.plot(Z,10**6*Z**(-3))
#圧縮部分運動エネルギースペクトル表示
    ax6.set_title("compressible kinetic energy spectrum")
    ax6.set_xscale('log')
    ax6.set_xlabel('k')
    ax6.set_yscale('log')
    ax6.set_ylabel('E_c(k)')
    #ax6.set_ylim(10**2,10**5)
    ax6.plot(Z,E_kin_compressible_spe)
    #ax6.plot(Z,Z**(-5/3))
    #ax6.plot(Z,10**6*Z**(-3))
#全運動エネルギースペクトル表示
    ax7.set_title("all kinetic energy spectrum")
    ax7.set_xscale('log')
    ax7.set_xlabel('k')
    ax7.set_yscale('log')
    ax7.set_ylabel('E(k)')
    #ax7.set_ylim(10**2,10**5)
    ax7.plot(Z,E_kin_spe)
    #ax7.plot(Z,Z**(-5/3))
    #ax7.plot(Z,10**6*Z**(-3))
#全非圧縮部分運動エネルギー
    ax8.set_title("incompressible kinetic energy")
    ax8.set_xlim(0,len(path_list)-1)
    ax8.set_xlabel('t (ħ/(gn))')
    ax8.set_ylabel('E_kin')
    ax8.plot(time,incompressible_kinetic_energy_list[0:i+1])
#全圧縮部分運動エネルギー
    ax9.set_title("compressilble kinetic energy")
    ax9.set_xlim(0,len(path_list)-1)
    ax9.set_xlabel('t (ħ/(gn))')
    ax9.set_ylabel('E_kin')
    ax9.plot(time,compressible_kinetic_energy_list[0:i+1])
#全運動エネルギー
    ax10.set_title("all kinetic energy")
    ax10.set_xlim(0,len(path_list)-1)
    ax10.set_xlabel('t (ħ/(gn))')
    ax10.set_ylabel('E_kin')
    ax10.plot(time,all_kinetic_energy_list[0:i+1])
    
    plt.savefig('output/summary/{}.png'.format(format(i, '04')))
    #plt.show()
    plt.close()
    
    del psi,E_kin_incompressible_spe,E_kin_compressible_spe
    del time,all_vortex_number,d,vortex_position_positive,vortex_position_negative

img=cv2.imread('output/summary/0000.png')
width=np.shape(img)[0]
height=np.shape(img)[1]

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('output/summary.mp4',fourcc, 20.0, (height, width))
for i in range(len(path_list)):
    img=cv2.imread('output/summary/%04d.png' % i)
    video.write(img)
video.release()

100%|██████████| 11/11 [00:27<00:00,  2.52s/it]
